<a href="https://colab.research.google.com/github/sophia-moore/232-Final-Project/blob/main/code_diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
diffusion_maps.py
-----------------
End-to-end helpers to

1. fit a Diffusion-Maps basis on training data           → Z_train   (n_train × k)
2. embed out-of-sample (test) observations by Nyström    → Z_test    (n_test  × k)
( sparse neighbourhood graph  +  symmetric eigenproblem )


Notation matches the description you supplied:

    • W_{ij}  =  κ(x_i , x_j)              (Gaussian kernel)
    • P       =  D⁻¹ W                    (row-stochastic transition matrix)
    • P Ψ   = Ψ Λ                         (right eigenvectors / eigenvalues)
    • Z_train = Ψ[:,1:K+1] · diag(Λ[1:K+1] ** (t/2))

    • For a new point  x* :
        w*_j      = κ(x*, x_j)
        p*_j      = w*_j / Σ_j w*_j
        φ_i(x*)   = (1 / λ_i) · Σ_j p*_j · φ_i(x_j)
        z_i(x*)   = λ_i ** (t/2) · φ_i(x*)

Dependencies
------------
numpy, scipy, scikit-learn  (for pairwise distances only)

"""

import numpy as np
import scipy.sparse as sp
import pandas as pd
from scipy.sparse.linalg import eigs
from scipy.sparse.linalg import eigsh              # sparse symmetric eigen-solver
from sklearn.metrics import pairwise_distances
from sklearn.neighbors import NearestNeighbors
from sklearn.utils.extmath import randomized_svd



In [ ]:
# --------------------------------------------------------------------- #
# 1. utilities                                                          #
# --------------------------------------------------------------------- #

def _bandwidth_median(distances_sq):
    """Median distance heuristic for σ (√median of squared dists)."""
    return np.sqrt(np.median(distances_sq))


def _symmetric_knn_rbf(X, n_neighbors=30, sigma='median', n_jobs=-1):
    """
    k-NN graph with RBF weights.  Returns W (csr) and sqrt(degrees).
    """
    nbrs = NearestNeighbors(
        n_neighbors=n_neighbors,
        metric="euclidean",
        algorithm="auto",
        n_jobs=n_jobs,
    ).fit(X)

    dists, idx = nbrs.kneighbors(X, return_distance=True)
    dists_sq = dists ** 2

    if sigma == "median":
        sigma = _bandwidth_median(dists_sq)
    sigma2 = sigma ** 2

    weights = np.exp(-dists_sq / sigma2)

    # Build sparse matrix
    n_samples = X.shape[0]
    row_idx = np.repeat(np.arange(n_samples), n_neighbors)
    W = sp.csr_matrix(
        (weights.ravel(), (row_idx, idx.ravel())),
        shape=(n_samples, n_samples),
    )

    # Symmetrise -- keep the *maximum* weight for each edge
    W = W.maximum(W.T)

    # Degrees and  D^{-1/2}
    deg = np.asarray(W.sum(axis=1)).ravel()
    inv_sqrt_deg = 1.0 / np.sqrt(np.clip(deg, 1e-12, None))

    return W, inv_sqrt_deg, sigma


def _normalized_graph_laplacian(W, inv_sqrt_deg):
    """
    Return the symmetric normalised affinity S = D^{-1/2} W D^{-1/2}.
    """
    D_inv_sqrt = sp.diags(inv_sqrt_deg)
    return D_inv_sqrt @ W @ D_inv_sqrt


In [ ]:
# --------------------------------------------------------------------- #
# 2. training phase                                                     #
# --------------------------------------------------------------------- #

def fit_diffusion_map(
    X_train,
    n_components=10,
    n_neighbors=30,
    sigma="median",
    t=2,
    n_jobs=-1,
    random_state=0,
):
    """
    Compute diffusion coordinates for the training set.
    Returns (λ,  Φ_train,  Z_train,  helper_dict)
    """
    W, inv_sqrt_deg, sigma = _symmetric_knn_rbf(
        X_train, n_neighbors, sigma, n_jobs
    )
    S = _normalized_graph_laplacian(W, inv_sqrt_deg)

    # eigsh on symmetric S  (largest algebraic eigenvalues)
    vals, vecs = eigsh(
        S,
        k=n_components + 1,          # +1 for trivial λ₁ = 1
        which="LA",
        tol=1e-4,
        maxiter=300,
    )

    # sort high → low
    order = np.argsort(-vals)
    vals, vecs = vals[order], vecs[:, order]

    # drop trivial component
    lam  = vals[1 : n_components + 1]
    phi  = vecs[:, 1 : n_components + 1]

    # diffusion coordinates
    Z = phi * lam ** (t / 2)

    helpers = dict(
        sigma=sigma,
        inv_sqrt_deg=inv_sqrt_deg,
        phi_train=phi,
        lam=lam,
        n_neighbors=n_neighbors,
        n_jobs=n_jobs,
    )
    return lam, phi, Z, helpers


# --------------------------------------------------------------------- #
# 3. Nyström out-of-sample                                              #
# --------------------------------------------------------------------- #

def nystrom_embed(
    X_test,
    X_train,
    helpers,
    t=2,
):
    """
    Embed X_test into the diffusion space learnt on X_train (Nyström).
    """
    sigma        = helpers["sigma"]
    n_neighbors  = helpers["n_neighbors"]
    n_jobs       = helpers["n_jobs"]
    phi_train    = helpers["phi_train"]   # shape (n_train, n_components)
    lam          = helpers["lam"]         # shape (n_components,)
    inv_sqrt_deg = helpers["inv_sqrt_deg"]

    # neighbor search
    nbrs = NearestNeighbors(
        n_neighbors=n_neighbors,
        metric="euclidean",
        algorithm="auto",
        n_jobs=n_jobs,
    ).fit(X_train)

    dists, idx = nbrs.kneighbors(X_test, return_distance=True)
    d2 = dists ** 2
    weights = np.exp(-d2 / (sigma ** 2))

    # normalize to one-step transition probabilities
    p_star = weights / weights.sum(axis=1, keepdims=True)  # shape (n_test, n_neighbors)

    # now compute Φ_test: weighted sum of phi_train over the neighbors
    #   for each test i:  Φ_test[i, :] = Σ_j p_star[i,j] * phi_train[ idx[i,j], : ]
    # we can do this in one vectorized step:
    #   gather the relevant phi_train rows,
    #   multiply by p_star[..., None], then sum over neighbour-axis.

    # shape (n_test, n_neighbors, n_components)
    phi_neigh = phi_train[idx]

    # weight & sum → (n_test, n_components)
    Phi_test = np.einsum("ij, ijk -> ik", p_star, phi_neigh)

    # Divide by eigenvalues and scale by lambda^(t/2)
    Phi_test = Phi_test / lam      # broadcast over columns
    Z_test = Phi_test * (lam ** (t / 2))

    return Z_test

In [ ]:
# def nystrom_embed(
#     X_test,
#     X_train,
#     helpers,
#     t=1,
# ):
#     """
#     Embed X_test into the diffusion space learnt on X_train (Nyström).
#     """
#     sigma        = helpers["sigma"]
#     n_neighbors  = helpers["n_neighbors"]
#     n_jobs       = helpers["n_jobs"]
#     phi_train    = helpers["phi_train"]
#     lam          = helpers["lam"]
#     inv_sqrt_deg = helpers["inv_sqrt_deg"]

#     # neighbour search train←test
#     nbrs = NearestNeighbors(
#         n_neighbors=n_neighbors,
#         metric="euclidean",
#         algorithm="auto",
#         n_jobs=n_jobs,
#     ).fit(X_train)

#     dists, idx = nbrs.kneighbors(X_test, return_distance=True)
#     dists_sq = dists ** 2
#     weights = np.exp(-dists_sq / (sigma ** 2))

#     # degree of each new node:  d* = Σ_j w*_j
#     d_star = np.sum(weights, axis=1, keepdims=True)
#     inv_sqrt_deg_star = 1.0 / np.sqrt(np.clip(d_star, 1e-12, None))

#     # normalised affinities  ŵ_{*j} / sqrt(d* d_j)
#     scale_train = inv_sqrt_deg[idx]      # (n_test, k)
#     A = (inv_sqrt_deg_star * weights) * scale_train

#     # Φ_i(x*) = (1 / λ_i) Σ_j A_{*j} Φ_i(x_j)
#     Phi_test = (A @ phi_train) / lam

#     Z_test = Phi_test * lam ** (t / 2)
#     return Z_test

In [ ]:
# --------------------------------------------------------------------- #
# 4. end-to-end helper                                                  #
# --------------------------------------------------------------------- #

def diffusion_maps_embed(
    X_train,
    X_test,
    n_components=10,
    n_neighbors=30,
    sigma="median",
    t=2,
    n_jobs=-1,
):
    """
    Full pipeline → Z_train, Z_test
    """
    lam, phi_train, Z_train, helpers = fit_diffusion_map(
        X_train,
        n_components,
        n_neighbors,
        sigma,
        t,
        n_jobs,
    )
    Z_test = nystrom_embed(
        X_test,
        X_train,
        helpers,
        t,
    )
    return Z_train, Z_test


In [ ]:
from pathlib import Path

# paste the fast_diffusion_maps.py code (or import it) -----------------------
# from fast_diffusion_maps import diffusion_maps_embed

# --- 1. File locations ------------------------------------------------------
csv_train = Path(r"C:\Users\gpapa\OneDrive\My Life\Education\2025 YALE MMS\Term_4S\T4_E_Adv_Lin_Algebra_p2\final_project\glove_train_avg.csv")

csv_test  = Path(r"C:\Users\gpapa\OneDrive\My Life\Education\2025 YALE MMS\Term_4S\T4_E_Adv_Lin_Algebra_p2\final_project\glove_test_avg.csv")

# --- 2. Load into NumPy -----------------------------------------------------
X_train = pd.read_csv(csv_train, index_col=0).to_numpy(dtype=np.float32)
X_test  = pd.read_csv(csv_test, index_col=0).to_numpy(dtype=np.float32)

print("train shape", X_train.shape, "   test shape", X_test.shape)

train shape (8378, 100)    test shape (2793, 100)


Below is a drop-in Jupyter cell that

1. loads your two TF-IDF matrices from the specified CSV paths,

2. builds a 40-nearest-neighbour Gaussian kernel graph,

3. keeps 10 latent diffusion factors,

4. embeds the 5 401 test documents by Nyström, and

5. writes the coordinates to Z_train.csv and Z_test.csv in the same folder as the source files.

In [ ]:
# --- 3. Diffusion-maps embedding -------------------------------------------
Z_train, Z_test = diffusion_maps_embed(
    X_train,
    X_test,
    n_components = 10,      # 10 latent factors
    n_neighbors  = 30,      # k-NN size (tune if graph too sparse/dense)
    sigma        = "median",# bandwidth heuristic  (see note below)
    t            = 2,       # horizon: emphasises local phrasing
    n_jobs       = -1,      # all cores for neighbour search
)


In [ ]:
print("train shape", Z_train.shape, "   test shape", Z_test.shape)

train shape (8378, 10)    test shape (2793, 10)


In [ ]:
# 5 Display
# -----------------------------------------------------------------
print("SIMPLE AVG w/o duplicates & t=2")
print("Z train matrix:")
print(Z_train[:1, :])
print('\n')
print("Z test matrix:")
print(Z_test[:1, :])

SIMPLE AVG w/o duplicates & t=2
Z train matrix:
[[-5.06514990e-04 -2.66675184e-04  3.11013587e-05  5.57686322e-02
  -7.77807169e-04 -1.36267789e-05  1.62279961e-06 -2.66268521e-07
  -1.43657075e-07  5.39011399e-02]]


Z test matrix:
[[-2.47476937e-04 -6.97087942e-05 -4.22419266e-06 -2.38777813e-03
   3.23332027e-04  2.68561307e-05 -1.68791842e-05  2.18382740e-05
  -2.58204695e-05 -7.03388138e-05]]


### TFIDF with duplicates & t=1

**Z train matrix:**  
**[[-0.00232007 -0.00115001 -0.00079325  0.00066468 -0.00046591  0.00035137 -0.00035245 -0.00036522 -0.00023834 -0.00022062]]**  
  
   
**Z test matrix:**  
**[[ 1.57580024e-03  1.30161322e-03  4.43076034e-03  3.55768305e-03 -6.54940916e-04  1.30642522e-03  7.73355226e-04  2.38318630e-04 7.85888666e-05  4.67890067e-05]]**  

### TFIDF w/o duplicates & t=1  
  
Z train matrix:  
[[ 0.01146238 -0.01153307  0.01145802 -0.01145705 -0.01151509  0.01152263
   0.01118894  0.01159457 -0.01139918  0.01139311]]


Z test matrix:  
[[ 0.01356615 -0.01365003  0.01356094 -0.01355976 -0.01362857  0.0136375
   0.01324168  0.01372175 -0.01349128  0.01348371]]

### TFIDF w/o duplicates & t=5  
  
Z train matrix:
[[ 0.01146233 -0.01153287  0.01145758 -0.01145626 -0.01151386  0.01152086
   0.01118658  0.01159138 -0.01139524  0.01138823]]


Z test matrix:
[[ 0.01356609 -0.0136498   0.01356042 -0.01355883 -0.01362711  0.01363539
   0.01323889  0.01371798 -0.01348661  0.01347793]]

### TFIDF w/o duplicates & t=2
Z train matrix:
[[-1.53445594e-03 -8.39407337e-06 -4.84700034e-05  5.85963664e-05
  -6.32784183e-05 -2.91732787e-04 -2.91255102e-04 -5.72971236e-02
  -5.78337815e-05 -5.01927656e-06]]


Z test matrix:
[[-1.00735881e-03 -4.54804495e-06 -2.82151424e-05  2.88756919e-05
  -2.71958870e-05 -1.70459974e-05 -1.34257639e-06  2.95148373e-03
   3.43587736e-05  2.84335936e-05]]

In [ ]:

# --- 4. Save results --------------------------------------------------------
out_dir = csv_train.parent
pd.DataFrame(
    Z_train,
    #index=pd.read_csv(csv_train, index_col=0).index,
    columns=[f"f{i+1}" for i in range(Z_train.shape[1])]
).to_csv(out_dir / "Z_train_diff_SIMPLE_t2.csv")

pd.DataFrame(
    Z_test,
    #index=pd.read_csv(csv_test,  index_col=0).index,
    columns=[f"f{i+1}" for i in range(Z_test.shape[1])]
).to_csv(out_dir / "Z_test_diff_SIMPLE_t2.csv")

print("✓  saved to", out_dir)


✓  saved to C:\Users\gpapa\OneDrive\My Life\Education\2025 YALE MMS\Term_4S\T4_E_Adv_Lin_Algebra_p2\final_project
